# Test to ensure modifications to function do not change evaluated value

Beginning modification of the function to handle case where N is not equal to 1.

In [1]:
import math
from numba import cuda
import ZMCIntegral
import time
import numpy as np
import scipy
import scipy.special
from scipy.integrate import quad
import helpers

Define constants in function

In [2]:
mu = 0.1  # Fermi-level
hOmg = 0.5  # Photon energy eV
a = 4  # AA
A = 4  # hbar^2/(2m)=4 evAA^2 (for free electron mass)
rati = 0.3  # ratio
eE0 = rati * ((hOmg) ** 2) / (2 * np.sqrt(A * mu))
# print(eE0)
Gamm = 0.005  # Gamma in eV.
KT = 1 * 10 ** (-6)
shift = A * (eE0 / hOmg) ** 2

The original function is declared below.

In [3]:
def Ds(kx, ky, qx, qy):
    N = 3
    dds = 0
    ds = 0
    ek = A * (math.sqrt((kx) ** 2 + (ky) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((kx + qx) ** 2 + (ky + qy) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((kx) ** 2 + (ky) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((kx + qx) ** 2 + (ky + qy) ** 2) / hOmg ** 2
    
    # print('xk = ', xk)
    # print('xkq = ', xkq)

    sing = np.arange(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
    taninv1kp = 2 * np.arctan2(Gamm, ek - hOmg / 2 + hOmg * sing)
    taninv1kqp = 2 * np.arctan2(Gamm, ekq - hOmg / 2 + hOmg * sing)
    taninv1km = 2 * np.arctan2(Gamm, ek + hOmg / 2 + hOmg * sing)
    taninv1kqm = 2 * np.arctan2(Gamm, ekq + hOmg / 2 + hOmg * sing)

    lg1kp = complex(0, 1) * np.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * sing) ** 2)
    lg1kqp = complex(0, 1) * np.log(Gamm ** 2 + (ekq - hOmg / 2 + hOmg * sing) ** 2)
    lg1km = complex(0, 1) * np.log(Gamm ** 2 + (ek + hOmg / 2 + hOmg * sing) ** 2)
    lg1kqm = complex(0, 1) * np.log(Gamm ** 2 + (ekq + hOmg / 2 + hOmg * sing) ** 2)

    ferp = np.heaviside(mu - hOmg / 2 - hOmg * sing, 0)
    ferm = np.heaviside(mu + hOmg / 2 - hOmg * sing, 0)

    dbl = np.arange(-(N - 1), (N - 1) + 1, 1)
    taninv2k = 2 * np.arctan2(Gamm, ek - mu + hOmg * dbl)
    taninv2kq = 2 * np.arctan2(Gamm, ekq - mu + hOmg * dbl)

    lg2k = complex(0, 1) * np.log(Gamm ** 2 + (ek - mu + hOmg * dbl) ** 2)
    lg2kq = complex(0, 1) * np.log(Gamm ** 2 + (ekq - mu + hOmg * dbl) ** 2)

    besk = scipy.special.jv(dbl, xk)
    beskq = scipy.special.jv(dbl, xkq)

    fac1 = ek - ekq + hOmg * dbl
    fac2 = fac1 + 2 * complex(0, 1) * Gamm
    fac3 = fac2 - ek + ekq
    
    # DEBUGGING
    # print('taninv2k = ', taninv2k)
    # print('taninv1kp = ', taninv1kp)
    # print('lg2k = ', lg2k)
    # print('lg1kp = ', lg1kp)
    # print('fac3 = ', fac3)
    # print('besk = ', besk)
    # print('beskq = ', beskq)
    
    for n in range(0, N):
        for alpha in range(0, N):
            for beta in range(0, N):
                for gamma in range(0, N):
                    for s in range(0, N):
                        for l in range(0, N):
                            p1p = fac1[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])
                            p2p = fac2[alpha - gamma + N - 1] * (
                                    taninv1kp[beta] - taninv2k[s + beta] + lg1kp[beta] - lg2k[s + beta])
                            p3p = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            p1m = fac1[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            p2m = fac2[alpha - gamma + N - 1] * (
                                    taninv1km[beta] - taninv2k[s + beta] + lg1km[beta] - lg2k[s + beta])

                            p3m = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            d1 = -2 * complex(0, 1) * fac1[beta - gamma + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     alpha - beta + N - 1]
                            
                            omint1p = ferp[s] * ((p1p + p2p + p3p) / d1)

                            omint1m = ferm[s] * ((p1m + p2m + p3m) / d1)

                            bess1 = beskq[gamma - n + N - 1] * beskq[gamma - l + N - 1] * besk[beta - l + N - 1] * besk[
                                beta - s + N - 1] * besk[alpha - s + N - 1] * besk[alpha - n + N - 1]

                            grgl = bess1 * (omint1p - omint1m)

                            pp1p = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            pp2p = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqp[beta] + taninv2kq[s + beta] + lg1kqp[beta] - lg2kq[
                                s + beta])

                            pp3p = fac3[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])

                            pp1m = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            pp2m = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqm[beta] + taninv2kq[s + beta] + lg1kqm[beta] - lg2kq[
                                s + beta])

                            pp3m = fac3[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            d2 = -2 * complex(0, 1) * fac1[alpha - beta + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     beta - gamma + N - 1]

                            omint2p = ferp[s] * ((pp1p + pp2p + pp3p) / d2)

                            omint2m = ferm[s] * ((pp1m + pp2m + pp3m) / d2)

                            bess2 = beskq[gamma - n + N - 1] * beskq[gamma - s + N - 1] * beskq[beta - s + N - 1] * \
                                    beskq[beta - l + N - 1] * besk[alpha - l + N - 1] * besk[alpha - n + N - 1]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + 2 * Gamm * (grgl + glga)
                            #DEBUG
                            # print('dds = ', dds)
                            # print('bess1=',  bess1)
    return dds.real

In [4]:
import helpers

Gammsq = Gamm ** 2
      
def modDsN2(x):
    N = 3
    dds = 0
    # ds = 0 # UNUSED
    qx = 0.1
    ek = A * (math.sqrt((x[0]) ** 2 + (x[1]) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((x[0] + qx) ** 2 + (x[1] + 0) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((x[0]) ** 2 + (x[1]) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((x[0] + qx) ** 2 + (x[1] + 0) ** 2) / hOmg ** 2

    singrmatrix = np.ones((6,N),dtype=np.float64)
    singzmatrix = np.ones((4,N),dtype=np.complex128)  

    n = 0
    i = -(N - 1) / 2
    for n in range(0, N):
        nu = hOmg * i
        chi = hOmg / 2
        omicron = ek - chi + nu
        phi = ekq - chi + nu
        iota = ek + chi + nu
        kappa = ekq + chi + nu

        omisq = omicron ** 2
        phisq = phi ** 2
        iotasq = iota ** 2
        kappasq = kappa ** 2

        singrmatrix[0,n] = 2 * math.atan2(Gamm, omicron)
        singrmatrix[1,n] = 2 * math.atan2(Gamm, phi)
        singrmatrix[2,n] = 2 * math.atan2(Gamm, iota)
        singrmatrix[3,n] = 2 * math.atan2(Gamm, kappa)

        singzmatrix[0,n] = complex(0, 1) * math.log(Gammsq + omisq)
        singzmatrix[1,n] = complex(0, 1) * math.log(Gammsq + phisq)
        singzmatrix[2,n] = complex(0, 1) * math.log(Gammsq + iotasq)
        singzmatrix[3,n] = complex(0, 1) * math.log(Gammsq + kappasq)

        chinu = chi - nu

        singrmatrix[4,n] = helpers.my_heaviside(mu - chinu)
        singrmatrix[5,n] = helpers.my_heaviside(mu + chinu)
        i = i + 1

    size_dbl = 5 # 2N-1
    dblrmatrix = np.ones((5,size_dbl),dtype=np.float64)
    dblzmatrix = np.ones((4,size_dbl),dtype=np.complex128)

    i = -(N-1)
    for n in range(0, size_dbl):
        xi = hOmg * i
        zeta = ek - mu + xi
        eta = ekq - mu + xi

        zetasq = zeta ** 2
        etasq = eta ** 2

        dblrmatrix[0,n] = 2 * math.atan2(Gamm, zeta)
        dblrmatrix[1,n] = 2 * math.atan2(Gamm, eta)

        logged1 = math.log(Gammsq + zetasq)
        logged2 = math.log(Gammsq + etasq)

        dblzmatrix[0,n] = complex(0, logged1)
        dblzmatrix[1,n] = complex(0, logged2)

        dblrmatrix[2,n] = helpers.besselj(i, xk)
        dblrmatrix[3,n] = helpers.besselj(i, xkq)

        fac1i = ek - ekq + xi
        fac2i = complex(fac1i, 2 * Gamm)
        dblrmatrix[4,n] = fac1i
        dblzmatrix[2,n] = fac2i
        dblzmatrix[3,n] = fac2i - ek + ekq
        i = i + 1
    
    # This is implementing what Mahmoud and I discussed to construct these outside.
    sdr00 = np.ones((N,N), dtype=np.float64)
    sdr20 = np.ones((N,N), dtype=np.float64)
    sdr11 = np.ones((N,N), dtype=np.float64)
    sdr31 = np.ones((N,N), dtype=np.float64)
    
    sdz00 = np.ones((N,N), dtype=np.complex128)
    sdz20 = np.ones((N,N), dtype=np.complex128)
    sdz11 = np.ones((N,N), dtype=np.complex128)
    sdz31 = np.ones((N,N), dtype=np.complex128)


    # Now to fill the oness, we use a dummy variable that will be referenced below by the actual indices.
    for i in range(0,N):  # i will be indexed by s below
        for j in range(0,N): # j will be indexed by alpha, beta, or gamma in the nested loops
            ij = i + j
            sdr00[i,j] = singrmatrix[0,j] - dblrmatrix[0,ij]
            sdr20[i,j] = singrmatrix[2,j] - dblrmatrix[0,ij]
            sdr11[i,j] = -singrmatrix[1,j] + dblrmatrix[1,ij]
            sdr31[i,j] = -singrmatrix[3,j] + dblrmatrix[1,ij]
            
            sdz00[i,j] = singzmatrix[0,j] + dblzmatrix[0,ij]
            sdz20[i,j] = singzmatrix[2,j] + dblzmatrix[0,ij]
            sdz11[i,j] = singzmatrix[1,j] + dblzmatrix[1,ij]
            sdz31[i,j] = singzmatrix[3,j] + dblzmatrix[1,ij]



    for n in range(0, N):
        nmod = n + N - 1

        for alpha in range(0, N):

            for beta in range(0, N):
                abdiff = alpha - beta + N - 1

                for gamma in range(0, N):
                    bgdiff = beta - gamma + N - 1
                    agdiff = alpha - gamma + N - 1

                    d1 = -2 * complex(0, 1) * dblrmatrix[4,bgdiff] * dblzmatrix[2,agdiff] * dblzmatrix[3,abdiff]
                    d2 = -2 * complex(0, 1) * dblrmatrix[4,abdiff] * dblzmatrix[2,agdiff] * dblzmatrix[3,bgdiff]

                    for s in range(0, N):
                        smod = s + N - 1
                        tau = s + beta
                        
                        p1p = dblrmatrix[4,bgdiff] * (sdr00[s,alpha] - sdz00[s,alpha])
                        p2p = dblzmatrix[2,agdiff] * (sdr00[s,beta] + singzmatrix[0,beta] - dblzmatrix[0,tau])
                        p3p = dblzmatrix[3,abdiff] * (sdr11[s,gamma] - sdz11[s,gamma])

                        p1m = dblrmatrix[4,bgdiff] * (sdr20[s,alpha] - sdz20[s,alpha])
                        p2m = dblzmatrix[2,agdiff] * (sdr20[s,beta] + singzmatrix[2,beta] - dblzmatrix[0,tau])
                        p3m = dblzmatrix[3,abdiff] * (sdr31[s,gamma] - sdz31[s,gamma])

                        omint1p = singrmatrix[4,s] * ((p1p + p2p + p3p) / d1)
                        omint1m = singrmatrix[5,s] * ((p1m + p2m + p3m) / d1)

                        pp1p = dblrmatrix[4,abdiff] * (sdr11[s,gamma] - sdz11[s,gamma])
                        pp2p = dblzmatrix[2,agdiff] * (sdr11[s,beta] + singzmatrix[1,beta] - dblzmatrix[1,tau])
                        pp3p = dblzmatrix[3,bgdiff] * (sdr00[s,alpha] - sdz00[s,alpha])

                        pp1m = dblrmatrix[4,abdiff] * (sdr31[s,gamma] - sdz31[s,gamma])
                        pp2m = dblzmatrix[2,agdiff] * (sdr31[s,beta] + singzmatrix[3,beta] - dblzmatrix[1,tau])
                        pp3m = dblzmatrix[3,bgdiff] * (sdr20[s,alpha] - sdz20[s,alpha])

                        omint2p = singrmatrix[4,s] * ((pp1p + pp2p + pp3p) / d2)
                        omint2m = singrmatrix[5,s] * ((pp1m + pp2m + pp3m) / d2)

                        for l in range(0, N):
                            lmod = l + N - 1

                            bess1 = dblrmatrix[3,gamma - nmod] * dblrmatrix[3,gamma - lmod] * dblrmatrix[2,beta - lmod] * dblrmatrix[2,beta - smod] * dblrmatrix[2,alpha - smod] * dblrmatrix[2,alpha - nmod]
                            
                            grgl = bess1 * (omint1p - omint1m)

                            bess2 = dblrmatrix[3,gamma - nmod] * dblrmatrix[3,gamma - smod] * dblrmatrix[3,beta - smod] * dblrmatrix[3,beta - lmod] * dblrmatrix[2,alpha - lmod] * dblrmatrix[2,alpha - nmod]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + Gamm * (grgl + glga)
    return dds.real 

In [5]:
orig = Ds(.01,.01,.01,0)

mod = modDsN2([0.1,0.1])
print('orig = ', orig)
print('mod = ', mod)
print('relerr = ', (mod-orig)/orig)

orig =  -5.162177692049824
mod =  1.1158135608842745
relerr =  -1.2161517149250245


# The modified function

The function is vectorized again to increase efficiency.

The custom Bessel function should now handle any-order*, first-kind Bessel function. *(Integer-order!)

NOTE: machine epsilon value ~E-16 implies that a relative error below E-15 is nonsense. The limit rests around here.

In [6]:
# Make error array.
relerror = np.zeros(256)

print('Comparing modified version and original')
print('================================================================================================')
print(' kx  | ky  | qx  | qy  | Ds          | modDs  ')
print('================================================================================================')
for i in range(0, 75, 10):
    for j in range(0, 75, 10):
        xin = [i/100, j/100]
        ds_result = Ds(i/100, j/100, 0.1, 0).real
        modds_result = modDsN2(xin)
        print('%2.1f  | %2.1f | 0.1 | 0   |'%(i/100, j/100), ds_result, ' | ', modds_result)
        relerror[i+j-1] = abs((modds_result / ds_result.real) - 1)
            

Comparing modified version and original
 kx  | ky  | qx  | qy  | Ds          | modDs  
0.0  | 0.0 | 0.1 | 0   | -40.257164577429265  |  -0.5935046977379085
0.0  | 0.1 | 0.1 | 0   | -99.66878453711495  |  -0.2161839241792475
0.0  | 0.2 | 0.1 | 0   | 98.11029272395426  |  5.5599503509302
0.0  | 0.3 | 0.1 | 0   | -0.9908017869406154  |  -3.976132030534118
0.0  | 0.4 | 0.1 | 0   | -8.712143331365649  |  -1.57967945564585
0.0  | 0.5 | 0.1 | 0   | -0.03932083674677117  |  -4.612463734494915
0.0  | 0.6 | 0.1 | 0   | -0.005169028564007705  |  -0.05874672582035488
0.0  | 0.7 | 0.1 | 0   | -0.0013483104379638628  |  -0.027082449184055862
0.1  | 0.0 | 0.1 | 0   | -31.724387674837  |  0.03441192777069602
0.1  | 0.1 | 0.1 | 0   | 33.98752697850443  |  1.1158135608842745
0.1  | 0.2 | 0.1 | 0   | 6.994460821711391  |  -1.4648299280600046
0.1  | 0.3 | 0.1 | 0   | -1.3796696374788808  |  -5.537902672501241
0.1  | 0.4 | 0.1 | 0   | -24.166778936935746  |  -4.0474189533013964
0.1  | 0.5 | 0.1 | 0   | -0.

In [7]:
errorsum = 0
print('Comparing modified version and original')
print('================================================================================================')
print(' kx  | ky  | qx  | qy  | rel error  ')
print('================================================================================================')
for i in range(0, 75, 10):
    for j in range(0, 75, 10):
        for k in range(1, 4, 1):
            print('%2.1f  | %2.1f | %2.1f | 0   |'%(i/100, j/100, k/10), relerror[i+j+k-1])
            errorsum = errorsum + relerror[i+j+k-1]
           
avgerror = errorsum / (256)            
print('================================================================================================')
print('The average error of modified function is ', avgerror)

Comparing modified version and original
 kx  | ky  | qx  | qy  | rel error  
0.0  | 0.0 | 0.1 | 0   | 0.0
0.0  | 0.0 | 0.2 | 0   | 0.0
0.0  | 0.0 | 0.3 | 0   | 0.0
0.0  | 0.1 | 0.1 | 0   | 0.0
0.0  | 0.1 | 0.2 | 0   | 0.0
0.0  | 0.1 | 0.3 | 0   | 0.0
0.0  | 0.2 | 0.1 | 0   | 0.0
0.0  | 0.2 | 0.2 | 0   | 0.0
0.0  | 0.2 | 0.3 | 0   | 0.0
0.0  | 0.3 | 0.1 | 0   | 0.0
0.0  | 0.3 | 0.2 | 0   | 0.0
0.0  | 0.3 | 0.3 | 0   | 0.0
0.0  | 0.4 | 0.1 | 0   | 0.0
0.0  | 0.4 | 0.2 | 0   | 0.0
0.0  | 0.4 | 0.3 | 0   | 0.0
0.0  | 0.5 | 0.1 | 0   | 0.0
0.0  | 0.5 | 0.2 | 0   | 0.0
0.0  | 0.5 | 0.3 | 0   | 0.0
0.0  | 0.6 | 0.1 | 0   | 0.0
0.0  | 0.6 | 0.2 | 0   | 0.0
0.0  | 0.6 | 0.3 | 0   | 0.0
0.0  | 0.7 | 0.1 | 0   | 0.0
0.0  | 0.7 | 0.2 | 0   | 0.0
0.0  | 0.7 | 0.3 | 0   | 0.0
0.1  | 0.0 | 0.1 | 0   | 0.0
0.1  | 0.0 | 0.2 | 0   | 0.0
0.1  | 0.0 | 0.3 | 0   | 0.0
0.1  | 0.1 | 0.1 | 0   | 0.0
0.1  | 0.1 | 0.2 | 0   | 0.0
0.1  | 0.1 | 0.3 | 0   | 0.0
0.1  | 0.2 | 0.1 | 0   | 0.0
0.1  | 0.2 | 0.2 | 0   |